# Reconnaissance d'entités nommées avec SpaCy

Preparation et chargement du corpus

In [1]:
from pathlib import Path
import re


data_dir = Path("../../data/txt")

all_txt = list(data_dir.glob("*.txt"))

YEAR = "1950"
pat = re.compile(rf"{YEAR}")

files_year = [p for p in all_txt if pat.search(p.name)]
print(f"Fichiers trouvés pour {YEAR}: {len(files_year)}")

corpus_year = ""
for path in files_year:
    for enc in ("utf-8", "latin-1", "cp1252"):
        try:
            with open(path, "r", encoding=enc, errors="ignore") as f:
                corpus_year += f.read() + "\n"
            break
        except Exception:
            continue

print(f"Taille du corpus {YEAR}: {len(corpus_year):,} caractères")
print("\nExtrait:\n", corpus_year[:500])

Fichiers trouvés pour 1950: 4
Taille du corpus 1950: 133,895 caractères

Extrait:
 L'AVENIR DU LUXEMBOURG Samedi 15 avri j 350, 
MORHET 
Soirée dramatique 
1 Le cercle dramatique Sainte-Cécile 
de Morhet reprendra, ce dimanche 16 
avril ^Quasimodo), sa brillante soirée 
qui a remporté un succès si remarqua-
| bie le 10 mars dernier. 
i Rappelons ie programme : 
; 1) ouverture : « Brabançonne »,par 
• la Fantare ; 2) « La .bohème », chœur 
à 2 voix exécuté par JV^.es Renée Cara, 
j Josée Goffin, Anyse Hubermont et Hé-
f lène Bellanger ; a) La comédie en deux 
actes de Marcell* 


La documentation est accessible ici: https://spacy.io/api

## Imports

In [4]:
!python -m spacy download fr_core_news_sm

     ---------------------------------------- 0.0/16.3 MB ? eta -:--:--
     ----- ---------------------------------- 2.1/16.3 MB 10.7 MB/s eta 0:00:02
     ---------- ----------------------------- 4.5/16.3 MB 11.1 MB/s eta 0:00:02
     ---------------- ----------------------- 6.8/16.3 MB 11.3 MB/s eta 0:00:01
     ---------------------- ----------------- 9.2/16.3 MB 11.4 MB/s eta 0:00:01
     ---------------------------- ---------- 11.8/16.3 MB 11.4 MB/s eta 0:00:01
     ---------------------------------- ---- 14.4/16.3 MB 11.6 MB/s eta 0:00:01
     --------------------------------------  16.3/16.3 MB 11.4 MB/s eta 0:00:01
     ---------------------------------------- 16.3/16.3 MB 10.9 MB/s  0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from collections import defaultdict
import spacy

try:
    nlp = spacy.load("fr_core_news_lg")
except OSError:
    nlp = spacy.load("fr_core_news_sm")

nlp.pipe_names

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

Analyse d'un petit extrait du corpus

In [8]:
text_ner = re.sub(r"-\s*\n\s*", "", corpus_year)
text_ner = re.sub(r"\s+\n", "\n", text_ner)
text_ner = re.sub(r"\n{2,}", "\n\n", text_ner)
len(text_ner)

130219

In [13]:
doc = nlp(text_ner[:50000])
entities = [(e.text.strip(), e.label_) for e in doc.ents]
len(entities), entities[:10]

(831,
 [('AVENIR', 'ORG'),
  ('Samedi', 'PER'),
  ('Soirée', 'LOC'),
  ('Sainte-Cécile', 'LOC'),
  ('Morhet', 'PER'),
  ('Brabançonne', 'LOC'),
  ('JV^.es Renée Cara', 'PER'),
  ('j Josée Goffin', 'LOC'),
  ('Héf lène Bellanger', 'PER'),
  ('Marcell', 'LOC')])

Collection et comptage des entités

In [16]:
from collections import Counter

def chunk(s, size=10000):
    for i in range(0, len(s), size):
        yield s[i:i+size]

entities = []
for piece in chunk(text_ner, 10000):
    d = nlp(piece)
    for e in d.ents:
        entities.append((e.text.strip(), e.label))
len(entities), entities[:10]

(2744,
 [('AVENIR', 383),
  ('Samedi', 4317129024397789502),
  ('Soirée', 385),
  ('Sainte-Cécile', 385),
  ('Morhet', 4317129024397789502),
  ('Brabançonne', 385),
  ('JV^.es Renée Cara', 4317129024397789502),
  ('j Josée Goffin', 385),
  ('Héf lène Bellanger', 4317129024397789502),
  ('Marcell', 385)])

In [18]:
entities = []
for piece in (text_ner[i:i+10000] for i in range(0, len(text_ner), 10000)):
    d = nlp(piece)
    for e in d.ents:
        entities.append((e.text.strip().replace("\n", ""), e.label_))
len(entities), entities[:5]

(2744,
 [('AVENIR', 'ORG'),
  ('Samedi', 'PER'),
  ('Soirée', 'LOC'),
  ('Sainte-Cécile', 'LOC'),
  ('Morhet', 'PER')])

In [19]:
def keep(txt):
    if len(txt) < 3: 
        return False
    if re.search(r"[~^@&%$<>]", txt): 
        return False
    if sum(ch.isdigit() for ch in txt) > 0: 
        return False
    return True

entities_clean = [(t, lab) for (t, lab) in entities if keep(t)]
len(entities_clean)

2490

In [20]:
from collections import Counter

top_per = Counter([t for t, lab in entities_clean if lab == "PER"]).most_common(10)
top_loc = Counter([t for t, lab in entities_clean if lab == "LOC"]).most_common(10)
top_org = Counter([t for t, lab in entities_clean if lab == "ORG"]).most_common(10)

print("Top PER :", top_per[:10])
print("Top LOC :", top_loc[:10])
print("Top ORG :", top_org[:10])

Top PER : [('Jambes', 15), ('Harold', 9), ('Oui', 7), ('Jeunes', 7), ('Phyllis', 7), ('Van Zeeland', 6), ('Roi', 6), ('Robert', 5), ('Jumet', 5), ('Mme de Sermaize', 4)]
Top LOC : [('Bruxelles', 27), ('Namur', 27), ('Belgique', 8), ('Etat', 8), ('Vieux', 8), ('Belga', 7), ('Paris', 6), ('Londres', 6), ('Charleroi', 6), ('Pologne', 6)]
Top ORG : [('NAMUR', 8), ('PRODENT', 5), ('BRUX', 4), ('XXV', 3), ('JAMBES', 3), ('Ford', 3), ('PAYS', 2), ("ministre de l'Intérieur", 2), ('Sénat', 2), ('Chambre', 2)]


In [4]:
nlp = spacy.load('fr_core_news_md')

## Exemple sur un corpus de test fourni par SpaCy

In [ ]:
# Imprimer le corpus de Spacy
sentences

In [ ]:
# Isoler la première phrase
sent = sentences[0]
sent

In [ ]:
# Traiter la phrase avec Spacy
doc = nlp(sent)

In [ ]:
type(doc)

In [ ]:
doc.text

In [ ]:
doc.to_json()

In [ ]:
# Appliquer le test sur toutes les phrases
for sent in sentences:
    doc = nlp(sent)
    entities = []
    for ent in doc.ents:
        entities.append(f"{ent.text} ({ent.label_})")
    if entities:
        print(f"'{doc.text}' contient les entités suivantes : {', '.join(entities)}")
    else:
        print(f"'{doc.text}' ne contient aucune entité")

## Appliquer la reconnaissance d'entités nommées sur notre corpus

In [ ]:
# Charger le texte
n=1000000
text = open("../data/all.txt", encoding='utf-8').read()[:n]

In [ ]:
%%time
# Traiter le texte

doc = nlp(text)

In [ ]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1

In [ ]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

Exercice: essayez de lister les lieux (LOC) et les organisations (ORG) les plus mentionnées dans le corpus